# Loading Kinoeva Data

Loading and parsing text data exported from Kinovea

This is the data that is exported as text format.  For example:

```
#Kinovea Trajectory data export
#T X Y
0:00:00:00 0.00 0.00 
0:00:00:03 1.00 9.00 
0:00:00:06 2.00 20.00 
0:00:00:10 3.00 34.00 
0:00:00:13 5.00 48.00 
0:00:00:16 6.00 66.00 
0:00:00:20 8.00 84.00 
0:00:00:23 8.00 104.00 
0:00:00:26 7.00 125.00 
0:00:00:30 6.00 145.00 
0:00:00:33 7.00 164.00 
0:00:00:36 9.00 182.00 
0:00:00:40 20.00 201.00 

```

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/2/CFF Demo_ Underhand Vertical Med Ball Throw.txt', delim_whitespace=True,
                 index_col=0)
df.head()

In [ ]:
# drop last column
df.drop(columns=['export'], inplace=True)
df.head()

In [ ]:
# replace column names
df.columns = ['X', 'Y']
df.head()

In [ ]:
# remove first row
df.drop(['#T'], inplace=True)
df.head()

In [ ]:
# rename index
df.index.name = 'T'
df.head()

In [ ]:
# save the original data as another column
df['Original_timestamp'] = df.index
df.index = pd.to_timedelta(df.index.str.replace(r'0:(.*)', r'\1'))
df.head()

In [ ]:
# If you want to convert to dates (non-offset)
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Period.strftime.html
#
# df.index = pd.to_datetime(df.index, format='0:%H:%M:%S', errors='coerce')
# pd.to_datetime(df.index, format='0:%H:%M:%S', errors='coerce')

In [ ]:
# The above is quite a bit of code, so maybe easier to organize in 
# a function for reusability

def read_kinoeva(input_file):
    """
    Arguments:
    input_file - path to the text file
    
    Returns:
    A pandas dataframe containing the data
    """
    # df will have local scope
    df = pd.read_csv(input_file, delim_whitespace=True, index_col=0)
    
    # drop last column
    df.drop(columns=['export'], inplace=True)
    
    # replace column names
    df.columns = ['X', 'Y']
    
    # remove first row
    df.drop(['#T'], inplace=True)
    
    # rename index
    df.index.name = 'T'
    
    # save the original data as another column
    df['Original_timestamp'] = df.index
    df.index = pd.to_timedelta(df.index.str.replace(r'0:(.*)', r'\1'))
    
    return df

In [ ]:
df = read_kinoeva('data/2/CFF Demo_ Underhand Vertical Med Ball Throw.txt')

print(df.info())
df.head()